## Задание:

Берем отзывы за лето (из архива с материалами или предыдущего занятия)

Учим conv сеть для классификации
Рассмотреть 2-а варианта сеточек:
Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 590.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8d00dd3e012a9d198817939bd7741c8f934a2847ed5bc4110f34c8155ee18bf3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import numpy as np
import pandas as pd

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_excel('/content/drive/MyDrive/nlp/les7/отзывы за лето.xls')

In [5]:
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [7]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [9]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [10]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [11]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:7]

['приложение', 'очень', 'удобно', 'работать', 'удобный', 'вс', 'спасибо']

In [12]:
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [13]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [14]:
data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']],
    dtype=np.int32)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

In [16]:
X_train.shape

(14461, 150)

In [17]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

le.classes_

array([1, 2, 3, 4, 5])

### 2.1 предобученные эмбединги

In [18]:
!wget http://vectors.nlpl.eu/repository/20/220.zip
!unzip /content/220.zip

--2024-04-03 08:52:19--  http://vectors.nlpl.eu/repository/20/220.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638171816 (609M) [application/zip]
Saving to: ‘220.zip’

220.zip             100%[===================>] 608.61M  22.2MB/s    in 28s     

2024-04-03 08:52:48 (22.0 MB/s) - ‘220.zip’ saved [638171816/638171816]

Archive:  /content/220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [19]:
embeddings_index = {}
with open('/content/model.txt') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 249334 word vectors.


In [20]:
embedding_dim = 200

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [21]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    weights=[embedding_matrix],
                    output_dim=embedding_dim,
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [22]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 34s 1s/step - loss: 1.5972 - accuracy: 0.6810 - val_loss: 1.5833 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 32s 1s/step - loss: 1.5715 - accuracy: 0.7051 - val_loss: 1.5577 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 28s 1s/step - loss: 1.5464 - accuracy: 0.7051 - val_loss: 1.5328 - val_accuracy: 0.7091
Epoch 4/10
26/26 [==============================] - 34s 1s/step - loss: 1.5220 - accuracy: 0.7051 - val_loss: 1.5086 - val_accuracy: 0.7091
Epoch 5/10
26/26 [==============================] - 29s 1s/step - loss: 1.4983 - accuracy: 0.7051 - val_loss: 1.4850 - val_accuracy: 0.7091
Epoch 6/10
26/26 [==============================] - 33s 1s/step - loss: 1.4753 - accuracy: 0.7051 - val_loss: 1.4622 - val_accuracy: 0.7091
Epoch 7/10
26/26 [==============================] - 28s 1s/step - loss: 1.4530 - accuracy: 0.7051 - val_loss: 1.4402 - val_accuracy: 0.7091
Epoch 8/10
26/26 [==

In [24]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 4s 336ms/step - loss: 1.3787 - accuracy: 0.7073
Test score: 1.3786698579788208
Test accuracy: 0.7073249220848083


### 2.2 эмбединги по умолчанию

In [25]:
model = Sequential()

model.add(Embedding(input_dim=max_words,
                    output_dim=128,
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [26]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_split=0.1)

Epoch 1/10
26/26 [==============================] - 21s 760ms/step - loss: 1.2961 - accuracy: 0.6876 - val_loss: 1.0365 - val_accuracy: 0.7091
Epoch 2/10
26/26 [==============================] - 19s 744ms/step - loss: 0.9675 - accuracy: 0.7051 - val_loss: 0.8564 - val_accuracy: 0.7091
Epoch 3/10
26/26 [==============================] - 20s 756ms/step - loss: 0.7393 - accuracy: 0.7407 - val_loss: 0.6800 - val_accuracy: 0.7664
Epoch 4/10
26/26 [==============================] - 20s 765ms/step - loss: 0.6274 - accuracy: 0.7796 - val_loss: 0.6492 - val_accuracy: 0.7685
Epoch 5/10
26/26 [==============================] - 20s 772ms/step - loss: 0.5786 - accuracy: 0.7897 - val_loss: 0.6381 - val_accuracy: 0.7795
Epoch 6/10
26/26 [==============================] - 21s 820ms/step - loss: 0.5420 - accuracy: 0.8081 - val_loss: 0.6314 - val_accuracy: 0.7816
Epoch 7/10
26/26 [==============================] - 20s 759ms/step - loss: 0.5084 - accuracy: 0.8230 - val_loss: 0.6324 - val_accuracy: 0.7823

In [28]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 159ms/step - loss: 0.6676 - accuracy: 0.7698
Test score: 0.6675931811332703
Test accuracy: 0.7697644233703613


Точность на предобученных эмбедингах ниже.